<a href="https://colab.research.google.com/github/andre5620/Hello-World/blob/master/Aula_05_An%C3%A1lise_n%C3%A3o_supervisionada_em_um_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trabalho realizado pelos alunos Marcelo de Fazio e Mario Estolano

## Entendimento do problema

O problema que abordamos refere-se a identificação de estados brasileiros que possuem curvas semelhantes de casos novos de infecção ao vírus COVID-19, a partir do primeiro caso identificado.
Utilizaremos as técnicas de agrupamento abordadas na aula, entre elas K-means e o agrupamento hieráquico aglomerativo.
Todo o código foi feito em python. 
Considerando que utilizamos a linguagem R em sala de aula, o trabalho propiciou uma maneira de praticarmos os conceitos ensinados em sala de aula, bem como a capacidade de aperfeiçoar as nossas técnicas e ferramentas de análise de dados utilizando outra linguagem(python).
O trabalho foi realizado de forma colaborativa, sendo desenvolvido e aperfeiçoado pelos dois integrantes do grupo em todas as etapas: na codificação, na análise e na avaliação dos resultados encontrados. 

## Entendimento dos dados

In [1]:
# bibliotecas necessárias para a exploração dos dados
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
# Para que possamos comparar desconsiderar o tempo como critério de semelhança utilizaremos a função do código enviado pela 
# professora,  para contabilizar a partir da janela do primeiro caso, adaptada para a nosssa base de dados
def create_ts(df):
    ts=df
    ts=ts.drop(['regiao','casosAcumulados', 'obitosNovos','obitosAcumulados'], axis=1)
    ts=ts.reset_index(drop=False)
    ts= ts.pivot(index="estado",columns= 'data', values='casosNovos').reset_index()
    ts.set_index('estado')
    ts=ts.T
    ts.columns=ts.loc['estado']
    ts=ts.drop('estado')
    ts=ts.fillna(0)
    ts=ts.reindex(sorted(ts.columns), axis=1)
    return (ts)

In [ ]:
import io
from google.colab import files
uploaded = files.upload()

In [ ]:
# leitura da base de dados no dia 08/05/2020
df = pd.read_csv("covid_08_05.csv",sep=";")

In [ ]:
# verificação da leitura dos dados do início do arquivo
df.head()

,regiao,estado,data,casosNovos,casosAcumulados,obitosNovos,obitosAcumulados
0,Norte,RO,2020-01-30,0,0,0,0
1,Norte,RO,2020-01-31,0,0,0,0
2,Norte,RO,2020-02-01,0,0,0,0
3,Norte,RO,2020-02-02,0,0,0,0
4,Norte,RO,2020-02-03,0,0,0,0


In [ ]:
# verificação da leitura dos dados do final do arquivo
df.tail()

,regiao,estado,data,casosNovos,casosAcumulados,obitosNovos,obitosAcumulados
2668,Centro-Oeste,DF,2020-05-03,83,1649,2,33
2669,Centro-Oeste,DF,2020-05-04,119,1768,0,33
2670,Centro-Oeste,DF,2020-05-05,50,1818,0,33
2671,Centro-Oeste,DF,2020-05-06,88,1906,1,34
2672,Centro-Oeste,DF,2020-05-07,352,2258,1,35


In [ ]:
# verificação os tipos dos dados
df.dtypes

regiao              object
estado              object
data                object
casosNovos           int64
casosAcumulados      int64
obitosNovos          int64
obitosAcumulados     int64
dtype: object

In [ ]:
# transformando a data em um objeto do tipo tempo
df['data'] = df['data'].astype('datetime64[ns]')

In [ ]:
# descrição da tabela
df.describe()

,casosNovos,casosAcumulados,obitosNovos,obitosAcumulados
count,2673.000000,2673.000000,2673.000000,2673.000000
mean,50.544706,690.875795,3.421624,44.888141
std,187.445668,2624.807049,14.876913,210.740571
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,6.000000,0.000000,0.000000
75%,26.000000,297.000000,1.000000,10.000000
max,3800.000000,39928.000000,224.000000,3206.000000


Vamos analisar os dados por regiãos para identificada

In [ ]:
#Modificações para tornar o layout mais legível do matplotlib
import locale
import matplotlib
#locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
#sns.set_style("white")
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.style"] = "normal"
plt.rcParams['font.size'] = 12
plt.rcParams['axes.formatter.use_locale'] = True
plt.rcParams['figure.figsize'] = [10, 5]
#0085bd
#297fd5

In [ ]:
fig = px.histogram(df, x="regiao", y="casosNovos", 
             template = 'plotly_white',  
             title="Quantidade de casos de COVID-19 por região do Brasil",
             histfunc="sum",
             color_discrete_sequence=['indianred'],
             opacity=0.95)
fig.update_traces(hovertemplate='Região: %{x} <br>Casos Novos: %{y}') 
fig.update_layout(xaxis_title="Região",
                  yaxis_title="",
                  xaxis={'categoryorder':'total descending'})
fig.show()

Podemos perceber que a maior ocorrência de óbitos se concentra na região sudeste

In [ ]:
fig = px.histogram(df, x="casosNovos", y="estado", 
             template = 'plotly_white',  
             title="Quantidade de casos de COVID-19 por região do Brasil",
             histfunc="sum",
             color_discrete_sequence=['indianred'],
             opacity=0.95, orientation='h')
fig.update_traces(hovertemplate='Estado: %{y} <br>Casos Novos: %{x}') 
fig.update_layout(xaxis_title="Estado",
                  yaxis_title="",
                  yaxis={'categoryorder':'total ascending'},
                  font=dict(
                    size=10,
                ))
fig.show()

Podemos perceber que a maior ocorrência de óbitos se concentra no estado de São Paulo

In [ ]:
import plotly.express as px

fig = px.line(df, x="data", y="casosNovos",
              facet_col= "estado", 
              hover_name="estado",
              log_y=True,
              color_discrete_sequence=['indianred'],
                  template = 'plotly_white'  )
fig.update_xaxes(matches='x',showticklabels=False,title_text="")
fig.update_yaxes(matches='y')
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("estado=", "")))
fig.update_layout( xaxis_title="",
                  yaxis_title="")
 
fig.show()

Utilizamos o log no eixo y  para identificar a taxa ao longo do tempo. No entanto, na análise exploratória ainda não deslocamos a janela da ocorrência para o primeiro caso. Na preparação dos dados, estamos interessados em entender como as curvas de crescimentos se relacionam entre os estados a partir do primeiro caso. Será que existem estados que possuem um comportamento semelhante nesta situação?

# Preparação dos dados

Nessa primeira etapa, gostaríamos de avaliar a semelhança da curva de crescimento a partir do primeiro caso, conforme dito anteriormente.
Utilizamos o código enviado pela professora para fazer um deslocamento dos dados e colocar todos os dados a partir do primeiro caso. Também realizamos a transposição da matriz de observações para preparação dos dados. Foi usado a normalização para que seja feita a redução de dimensionalidade(PCA) de forma adequada considerando a quantidade de componentes principais necessárias para uma boa explicação da variabilidade dos dados

In [ ]:
# criação de TimeSeries
ts=create_ts(df)

In [ ]:
# Adicionando todos as times series de cada estado em um unico timeseries
appended_data = []
for i, est in enumerate(ts.columns):
    ts_temp = ts[est]
    ts_temp = ts_temp[ts_temp > 0]
    ts_temp = ts_temp.reset_index(drop=True)
    appended_data.append(ts_temp)
ts_all_limited = pd.concat(appended_data, axis=1, sort=False,join='inner')    
ts_all_limited.head()

,AC,AL,AM,AP,BA,CE,DF,ES,GO,MA,MG,MS,MT,PA,PB,PE,PI,PR,RJ,RN,RO,RR,RS,SC,SE,SP,TO
0,3,1,1,1,1,5,1,1,3,1,1,2,1,1,1,2,3,6,1,1,1,2,1,2,1,1,1
1,4,3,2,1,1,4,1,8,3,1,1,2,1,1,1,14,1,7,1,5,2,6,1,2,3,1,1
2,2,1,8,2,1,11,4,2,2,6,3,3,5,2,1,12,2,10,1,3,2,2,2,2,1,1,3
3,2,2,15,4,27,35,2,2,4,2,2,2,1,1,2,2,2,9,5,4,1,2,2,1,1,3,2
4,6,2,6,2,3,13,5,13,3,3,8,7,3,2,4,7,1,11,5,1,2,4,4,3,4,4,1


In [ ]:
ts_all_limited.shape

(34, 27)

In [ ]:
# transpondo a matriz para colocar no modelo
ts_all_limited = ts_all_limited.T

In [ ]:
X = ts_all_limited.values

In [ ]:
# Normalização dos dados
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
import numpy as np
import plotly.graph_objects as go
import matplotlib.ticker as ticker
from sklearn.decomposition import PCA

pca = PCA().fit(X)

fig = go.Figure()

fig.add_trace(go.Scatter(y=np.cumsum(pca.explained_variance_ratio_),x=np.arange(1,28),marker_color='indianred'))
fig.update_layout( xaxis_title="Número de componentes",
                  yaxis_title="Variância explicativa acumulada",
                 title="Gráfico cotovelo",
                 template = 'plotly_white')

fig.update_xaxes(tick0=1, dtick=1)

A partir de 5 componentes principais já é capaz de explicar em torno de 80% da variabilidade dos dados.
Utilizaremos 5 componentes principais conforme código abaixo.

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=)
principalComponents = pca.fit_transform(X)
principalDf = pd.DataFrame(data = principalComponents)

Componentes principais criados

In [ ]:
principalDf.head()

,0,1,2,3,4
0,-3.377690,-0.191452,1.546599,-0.887009,0.605812
1,-2.213912,1.594832,0.823937,-0.264923,-1.175346
2,5.651906,-1.627794,-0.708536,1.598379,0.294760
3,-0.308008,-0.289457,-2.850630,-0.185642,-1.532167
4,0.561306,-1.013463,-1.437188,-0.749018,-1.127750


In [ ]:
principalDf.shape

(27, 5)

Dias mais importantes para separação dos dados

In [ ]:
pd.DataFrame(pca.components_)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,0.053038,0.050934,0.142634,0.145805,0.116861,0.143906,0.154219,0.076540,0.058535,0.150794,0.160543,0.090352,0.188285,0.046940,0.107989,0.221639,0.222294,0.218230,0.198140,0.231027,0.225018,0.167954,0.206093,0.221018,0.205795,0.212879,0.153248,0.150331,0.183618,0.171037,0.205968,0.220076,0.227351,0.177016
1,-0.216847,0.002940,-0.168427,-0.243768,-0.182260,-0.310210,-0.268978,-0.259664,-0.196498,-0.212737,-0.229537,-0.119564,0.074559,-0.162232,-0.066669,0.028235,-0.080892,0.026397,-0.147937,0.113417,-0.088041,0.078887,-0.166290,0.119258,0.176583,0.196278,0.006904,0.057826,0.229211,0.256058,0.223752,0.189407,0.136174,0.120251
2,0.318885,0.401647,0.300544,0.033240,0.250345,0.074543,0.054908,-0.235708,-0.197515,-0.004469,-0.116947,-0.195345,-0.253927,-0.384238,-0.326224,-0.135456,0.010438,-0.154494,0.021646,0.030313,-0.016054,0.120320,0.156056,0.095591,-0.032587,0.076995,-0.006941,-0.000500,-0.045828,0.001855,-0.026658,-0.002172,0.022649,0.083905
3,-0.156635,0.258090,0.197516,-0.194780,-0.048633,-0.148045,-0.195339,0.166499,0.082298,0.083525,0.018675,-0.080419,-0.190500,0.166949,0.291686,-0.072129,-0.027566,-0.073900,-0.057058,-0.008155,-0.085711,0.376824,-0.014280,0.010625,0.139627,0.105007,0.350256,0.384280,-0.195937,-0.111180,-0.146175,-0.126113,-0.124676,0.055572
4,0.023282,0.164412,0.076612,-0.194643,0.241917,-0.022004,-0.154839,-0.047896,0.496841,0.273696,-0.193923,-0.209448,0.185499,0.186783,-0.083884,0.081776,-0.130654,-0.018600,0.109255,0.162927,-0.091522,0.143355,-0.030078,0.020472,0.045914,-0.014209,-0.176828,-0.262784,0.175064,0.145917,-0.004882,-0.001148,-0.136847,-0.312512


# Modelagem: 

Utilizamos o KMeans e o Hieráquico para análise.
No Kmeans avaliaremos até 15 possibilidades de grupos.
Por meio do gráfico cotovelo, também definimos a quantidade ótima de grupos.
Visualmente, pode ser feito olhando a variação da inclinação do WCSS.

In [ ]:
from sklearn.cluster import KMeans
wcss = []
for i in range(1, 15):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(principalDf)
    wcss.append(kmeans.inertia_)

fig = go.Figure()
fig.add_trace(go.Scatter(y=wcss,x=np.arange(1,15),marker_color='indianred'))
fig.update_layout( xaxis_title="Número de clustes",
                  yaxis_title="WCSS",
                 title="Gráfico cotovelo",
                 template = 'plotly_white')

A partir de 5 grupos a variação do WCSS começa a ter uma variação menor. 
Portanto utilizaremos 5 grupos no algoritmo Kmeans. 
Esse algoritmo gera centróides aleatórios e calcula a média de cada centróide a cada interação

In [ ]:
X = principalDf.values

In [ ]:
# Fitting K-Means to the dataset
kmeans = KMeans(n_clusters = 5, init = 'k-means++', random_state = 42).fit(X)
y_kmeans = kmeans.labels_

resultado = pd.DataFrame(np.vstack((np.array(ts_all_limited.index.values),np.array(y_kmeans)))).T
resultado.columns = ['estado','grupo']
resultado["PCA1"] = X[:,0]
resultado["PCA2"] = X[:,1]

fig = go.Figure()
# Visualising the clusters

fig = px.scatter(resultado,x="PCA1",y="PCA2",text="estado",color="grupo")
#fig.add_trace(go.Scatter(y=kmeans.cluster_centers_[:, 1],x=kmeans.cluster_centers_[:, 0],marker_color='yellow',
#                        mode='markers',
#                      legendgroup="asadsd"))
fig.update_traces(marker=dict(size=20))
fig.update_layout( xaxis_title="Componente principal 1",
                  yaxis_title="Componente principal 2",
                 title="Gráfico de aglomeração",
                 template = 'plotly_white')

Observa-se que foram criados 6 grupos, sendo três deles com apenas um estado. São eles Ceará, Pernambuco e São Paulo.

Verificando a quantidade de elementos por classe

In [ ]:
grouped = resultado.groupby('grupo')
grouped["grupo"].count()

grupo
0    13
1     1
2     4
3     1
4     8
Name: grupo, dtype: int64

In [ ]:
import plotly.figure_factory as ff
ff.create_dendrogram(X, orientation='bottom',labels=ts_all_limited.index.values)

Poderíamos definir a quantidade de grupos: por threshold específico, pela análise do negócio ou escolhendo a maior distância 
entre grupos e dividir por 2. Utilizaremos como padrão, a definição do algoritmo que informa que a quantidade seria de 3 clusters

In [ ]:
# Fitting Hierarchical Clustering to the dataset
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn.cluster import AgglomerativeClustering
clustering = AgglomerativeClustering(n_clusters = 3, affinity = 'euclidean', linkage = 'ward').fit(X)
clustering.labels_
y_kmeans = clustering.labels_
resultado = pd.DataFrame(np.vstack((np.array(ts_all_limited.index.values),np.array(y_kmeans)))).T
resultado.columns = ['estado','grupo']
resultado["PCA1"] = X[:,0]
resultado["PCA2"] = X[:,1]

fig = go.Figure()
# Visualising the clusters

fig = px.scatter(resultado,x="PCA1",y="PCA2",text="estado",color="grupo")
#fig.add_trace(go.Scatter(y=kmeans.cluster_centers_[:, 1],x=kmeans.cluster_centers_[:, 0],marker_color='yellow',
#                        mode='markers',
#                      legendgroup="asadsd"))
fig.update_traces(marker=dict(size=20))
fig.update_layout( xaxis_title="Componente principal 1",
                  yaxis_title="Componente principal 2",
                 title="Gráfico de aglomeração",
                 template = 'plotly_white')

# Avaliação

Para descobrir o melhor método sem ser visualmente, utilizaremos a métrica da silhueta e análise do gráfico, comparando até 7 agrupamentos do Kmeans e do HC. O coeficiente da silhueta informa a média da similiraridade dos grupos enquanto considera também a distância entre grupos distintos.

#### Kmeans

No gráfico abaixo geramos o gráfico da silhueta de 2 até 7 grupos no método Kmeans.Podemos perceber que o melhor resultado encontrado para o algoritmo Kmeans foi utilizando 6 grupos, pois foi o maior valor do coeficiente de silhueta e não encontramos nenhum valor negativo para o coeficiente de silhueta, indicando que ocorreu um bom agrupamento.

In [ ]:
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from plotly.subplots import make_subplots
import plotly
range_n_clusters = [2,3,4,5,6,7]

colors = plotly.colors.DEFAULT_PLOTLY_COLORS
import matplotlib.cm as cm
for n_clusters in range_n_clusters:
    # Criando um subplot com 1 linha e 2 colunas
    fig = make_subplots(rows=1, cols=2)        
    
    fig.update_xaxes(tickvals=[-0.1, 0, 0.5, 1],range=[-0.1,1],
    selector=dict(type="scatter", mode="lines"))
    fig.update_yaxes(range=[0, len(X) + (n_clusters + 1) * 10],
    selector=dict(type="scatter", mode="lines"))
    fig.update_layout(showlegend=False)
    
    
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)    
    cluster_labels = clusterer.fit_predict(X)
    
    # Calculando o valor da silhueta
    silhouette_avg = metrics.silhouette_score(X, cluster_labels)
    print("Kmeans para n_grupos =", n_clusters,"A médias da  silhueta é :", silhouette_avg)
    fig.update_layout(shapes=[                  
        dict(
            type= 'line',
            yref= 'paper', y0= 0, y1= 1,
            xref= 'x', x0= silhouette_avg, x1= silhouette_avg,
            line=dict(
                color="Red",
                width=2,
                dash="dashdot",
            )
        )])  
    # Compute the silhouette scores for each sample
    sample_silhouette_values = metrics.silhouette_samples(X, cluster_labels)
    
    fig.update_layout( xaxis_title="O valor do coeficiente da silhueta",
                  yaxis_title="Observações da silhueta de vários clusters",
                 title="Gráfico da silhueta",
                 template = 'plotly_white')
    fig.update_xaxes(title_text="PCA1", row=1, col=2)
    fig.update_yaxes(title_text="PCA2", row=1, col=2)
    y_lower = 10
  
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        
        fig.add_trace(go.Scatter(x=ith_cluster_silhouette_values,y=np.arange(y_lower, y_upper)
                                 , fill='tozerox', mode = 'lines',
                                 line=dict(
                                      color=colors[i]
                                 )), row=1, col=1)         
        y_lower = y_upper + 5  # 10 for the 0 samples
        
        resultado = pd.DataFrame(np.vstack((np.array(ts_all_limited.index.values),np.array(cluster_labels)))).T
        resultado.columns = ['estado','grupo']
        resultado["PCA1"] = X[:,0]
        resultado["PCA2"] = X[:,1]               
        resultado = resultado[resultado["grupo"]==i]

        fig2 = go.Scatter(x = resultado["PCA1"].values, y= resultado["PCA2"].values,
                          mode="markers+text",
                          text = resultado["estado"].values,
                          marker=dict(
                                size=18,
                                color=colors[i],
                                line=dict(
                                    color='DarkSlateGrey',
                                    width=1
                                )
                            ))
        fig.add_trace( fig2, row=1, col=2)
    fig.show()

Kmeans para n_grupos = 2 A médias da  silhueta é : 0.5012421227508266


Kmeans para n_grupos = 3 A médias da  silhueta é : 0.3625586852658401


Kmeans para n_grupos = 4 A médias da  silhueta é : 0.37913226370147685


Kmeans para n_grupos = 5 A médias da  silhueta é : 0.3940445395643347


Kmeans para n_grupos = 6 A médias da  silhueta é : 0.3996132229699136


Kmeans para n_grupos = 7 A médias da  silhueta é : 0.37900166343069125


### HC

Também realizamos o gráfico da silhueta de 2 até 7 grupos no método HC.Podemos perceber que o melhor resultado encontrado para o algoritmo HC foi utilizando 3 grupos, pois foi o maior valor do coeficiente de silhueta e não existe nenhuma observação com valor do coeficiente  negativo nesse caso. Comparado ao Kmeans, o HC apresentou um melhor resultado na média do coeficiente de silhueta, portanto conseguiu agrupar melhor as observações do mesmo grupo e com a melhor separação entre grupos.

In [ ]:
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from plotly.subplots import make_subplots
import plotly
range_n_clusters = [2,3,4,5,6,7]

colors = plotly.colors.DEFAULT_PLOTLY_COLORS
import matplotlib.cm as cm
for n_clusters in range_n_clusters:
    # Criando um subplot com 1 linha e 2 colunas
    fig = make_subplots(rows=1, cols=2)        
    
    fig.update_xaxes(tickvals=[-0.1, 0, 0.5, 1],range=[-0.1,1],
    selector=dict(type="scatter", mode="lines"))
    fig.update_yaxes(range=[0, len(X) + (n_clusters + 1) * 10],
    selector=dict(type="scatter", mode="lines"))
    fig.update_layout(showlegend=False)
    
    clusterer = AgglomerativeClustering(n_clusters = n_clusters, affinity = 'euclidean', linkage = 'ward')
    cluster_labels = clusterer.fit_predict(X)
    
    # Calculando o valor da silhueta
    silhouette_avg = metrics.silhouette_score(X, cluster_labels)
    print("O agrupmento para n_grupos =", n_clusters,"A médias da  silhueta é :", silhouette_avg)
    fig.update_layout(shapes=[                  
        dict(
            type= 'line',
            yref= 'paper', y0= 0, y1= 1,
            xref= 'x', x0= silhouette_avg, x1= silhouette_avg,
            line=dict(
                color="Red",
                width=2,
                dash="dashdot",
            )
        )])  
    # Compute the silhouette scores for each sample
    sample_silhouette_values = metrics.silhouette_samples(X, cluster_labels)
    
    fig.update_layout( xaxis_title="O valor do coeficiente da silhueta",
                  yaxis_title="Observações da silhueta de vários clusters",
                 title="Gráfico da silhueta",
                 template = 'plotly_white')
    fig.update_xaxes(title_text="PCA1", row=1, col=2)
    fig.update_yaxes(title_text="PCA2", row=1, col=2)
    y_lower = 10
  
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        
        fig.add_trace(go.Scatter(x=ith_cluster_silhouette_values,y=np.arange(y_lower, y_upper)
                                 , fill='tozerox', mode = 'lines',
                                 line=dict(
                                      color=colors[i]
                                 )), row=1, col=1)         
        y_lower = y_upper + 5  # 10 for the 0 samples
        
        resultado = pd.DataFrame(np.vstack((np.array(ts_all_limited.index.values),np.array(cluster_labels)))).T
        resultado.columns = ['estado','grupo']
        resultado["PCA1"] = X[:,0]
        resultado["PCA2"] = X[:,1]               
        resultado = resultado[resultado["grupo"]==i]

        fig2 = go.Scatter(x = resultado["PCA1"].values, y= resultado["PCA2"].values,
                          mode="markers+text",
                          text = resultado["estado"].values,
                          marker=dict(
                                size=18,
                                color=colors[i],
                                line=dict(
                                    color='DarkSlateGrey',
                                    width=1
                                )
                            ))
        fig.add_trace( fig2, row=1, col=2)
    fig.show()
    

O agrupmento para n_grupos = 2 A médias da  silhueta é : 0.5012421227508266


O agrupmento para n_grupos = 3 A médias da  silhueta é : 0.48638046171308863


O agrupmento para n_grupos = 4 A médias da  silhueta é : 0.36411195433131843


O agrupmento para n_grupos = 5 A médias da  silhueta é : 0.3801435812066816


O agrupmento para n_grupos = 6 A médias da  silhueta é : 0.3872438388531633


O agrupmento para n_grupos = 7 A médias da  silhueta é : 0.3829551117131214


## Recomendação

Vale ressaltar que o estado do Ceará, no método HC, foi o único estado que ficou em um único grupo, separado do restante. Na análise explorátoria, já foi possível identificar que a curva de crescimento desse estado está alta quando comparada com os outros estados. Nesse caso, parece que as medidas adotadas no Ceará poderiam ser revistas, para que se reduza o crescimento de novos casos, por meio de medidas mais restritivas, tais como lockdown, que poderiam reduzir a quantidade de casos novos entre as pessoas daquele estado.